In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import torchtext
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import random
import numpy as np
from torchtext.legacy.data import Field,LabelField,TabularDataset,BucketIterator
import re
# from unidecode import unidecode
# import contractions
import spacy
import seaborn as sns
from torchtext.data.utils import get_tokenizer
from torchtext.legacy.data import BucketIterator,Field,LabelField,TabularDataset
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
dirs="/content/drive/MyDrive/dataset/Corona_virus_nlp/"

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/dataset/Corona_virus_nlp/Corona_Virus_Nlp/Rnn_data.csv")
data.head()

,Sentiment,Text
0,1,advice talk to your neighbours family to excha...
1,1,coronavirus australia woolworths to give elder...
2,1,my food stock is not the only one which is emp...
3,0,me ready to go at supermarket during the c...
4,1,as news of the regionas first confirmed covid1...


In [ ]:
tokenizer=get_tokenizer("basic_english")

In [ ]:
Text=Field(tokenize=tokenizer,lower=True,include_lengths=True)
Label=LabelField(dtype=torch.float)

In [ ]:
field=[("label",Label),("text",Text)]
field

[('label', <torchtext.legacy.data.field.LabelField at 0x7f8a5e9de810>),
 ('text', <torchtext.legacy.data.field.Field at 0x7f8a50ff49d0>)]

In [ ]:
training_data=TabularDataset(path="/content/drive/MyDrive/dataset/Corona_virus_nlp/Corona_Virus_Nlp/Rnn_data.csv",format="csv",
                             skip_header=True,fields=field)

print(vars(training_data[3]))

{'label': '0', 'text': ['me', 'ready', 'to', 'go', 'at', 'supermarket', 'during', 'the', 'covid19', 'outbreak', 'not', 'because', 'i', 'am', 'paranoid', 'but', 'because', 'my', 'food', 'stock', 'is', 'litteraly', 'empty', 'the', 'coronavirus', 'is', 'a', 'serious', 'thing', 'but', 'please', 'do', 'not', 'panic', 'it', 'causes', 'shortage', 'coronavirusfrance', 'restezchezvous', 'stayathome', 'confinement']}


In [ ]:
train_data,valid_data=training_data.split(split_ratio=0.7,random_state=random.seed(2020))

In [ ]:
MAX_VOCAB_SIZE = 25_000
Text.build_vocab(train_data,max_size = MAX_VOCAB_SIZE,vectors = "glove.6B.100d",unk_init = torch.Tensor.normal_)
Label.build_vocab(train_data)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_iter,valid_iter=BucketIterator.splits((train_data,valid_data),
                                    sort_key=lambda x:len(x.text),
                                    batch_size=64,
                                    sort_within_batch=True,
                                    device=device)

In [ ]:
for i in train_iter:
  print(i.text)
  break

(tensor([[ 1963,    14,  2081,  ...,    14,     7,  1587],
        [   23,  2142,    52,  ...,    98,  7580,  6554],
        [ 1307,     3,  2073,  ...,    83,   681,     3],
        ...,
        [ 1933,  5237,     6,  ...,    71,   173,   172],
        [  106,  1470,   830,  ...,   114,    48,     5],
        [ 1733,  6647, 10610,  ...,   457,    96,   262]]), tensor([33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33]))


In [ ]:
Text.vocab.stoi[Text.pad_token]

1

In [ ]:
Text.pad_token

'<pad>'

In [ ]:
class LSTM(nn.Module):
  def __init__(self,vocab_size,embedding_dim,hidden_dim,output_layer,n_layer,dropout,pad_idx):
    super(LSTM,self).__init__()
    self.n_layer=n_layer
    self.hidden_dim=hidden_dim
    self.dropout=nn.Dropout(dropout)


        
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
    self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=True, 
                           dropout=dropout)
        
    self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    self.dropout = nn.Dropout(dropout)
    # self.sigmoid=nn.Sigmoid()

  # def init_hidden(self,batch_size):
  #   return (torch.autograd.Variable(torch.zeros(self.n_layer*2,batch_size,self.hidden_dim)),
  #           torch.autograd.Variable(torch.zeros(self.n_layer*2,batch_size,self.hidden_dim)))
    

  def forward(self,text,text_len):
    for i, _ in enumerate(text_len):
        if text_len[i] == 0:
          text_len[i] = 1
    #text [batch_size,text_len]
    #text_len=[text_len]
    # batch_size=text.size(0)
    x=self.dropout(self.embedding(text))

    # h0,c0=self.init_hidden(batch_size)
    pack_packed_sequence=nn.utils.rnn.pack_padded_sequence(x,text_len)

    packed_output,(hidden,cell)=self.lstm(pack_packed_sequence)
    #unpack sequence
    output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
    hidden =self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
    # hidden=hidden[:, -1, :]
    # hidden=torch.tanh(hidden)


    fc=self.fc(hidden)

    # fc2=self.linear2(fc)

    # dense_output=self.sigmoid(fc2)

    return fc

In [ ]:
vocab_size=len(Text.vocab)
embedding_dim=100
hidden_dim=256
n_layers=2
output_dim=1
dropout=0.5
pad_token=Text.vocab.stoi[Text.pad_token]
model=LSTM(vocab_size,embedding_dim,hidden_dim,output_dim,n_layers,dropout,pad_token)
model

LSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(25002, 100, padding_idx=1)
  (lstm): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
for data in train_iter:
  text,text_len=data.text
  print(model(text,text_len))
  break

tensor([[ 0.0288],
        [-0.0139],
        [ 0.0549],
        [ 0.0248],
        [ 0.0304],
        [-0.0227],
        [ 0.0919],
        [ 0.0483],
        [-0.0346],
        [ 0.0412],
        [-0.0336],
        [-0.0681],
        [ 0.0179],
        [-0.0227],
        [-0.0088],
        [ 0.0134],
        [-0.0124],
        [ 0.0569],
        [-0.0264],
        [ 0.0365],
        [-0.0457],
        [ 0.0261],
        [ 0.0034],
        [ 0.0401],
        [ 0.0290],
        [-0.0340],
        [ 0.0230],
        [ 0.0893],
        [ 0.0233],
        [ 0.0167],
        [-0.0080],
        [ 0.0598],
        [ 0.0688],
        [ 0.0082],
        [-0.0214],
        [-0.0013],
        [ 0.0106],
        [ 0.1236],
        [ 0.0210],
        [-0.0026],
        [ 0.0509],
        [ 0.0128],
        [ 0.0203],
        [-0.0571],
        [ 0.0199],
        [ 0.0420],
        [-0.0421],
        [ 0.0140],
        [ 0.0275],
        [ 0.0550],
        [ 0.0708],
        [ 0.0214],
        [-0.

In [ ]:
pretrained_embeddings = Text.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [ ]:

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.9174,  1.7505, -0.6514,  ...,  0.4704,  0.3767,  0.6201],
        [ 0.2355, -1.7910,  0.3254,  ..., -0.0284, -1.3602,  1.6034],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.6640,  0.7606, -1.8337,  ...,  0.3142,  0.8558,  0.3032],
        [ 0.4706,  1.0778, -0.2694,  ..., -0.7763, -1.0243,  0.0688],
        [ 0.2278, -0.0096, -0.3427,  ...,  0.5979,  0.2396,  0.4793]])

In [ ]:

UNK_IDX = Text.vocab.stoi[Text.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[pad_token] = torch.zeros(embedding_dim)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.6640,  0.7606, -1.8337,  ...,  0.3142,  0.8558,  0.3032],
        [ 0.4706,  1.0778, -0.2694,  ..., -0.7763, -1.0243,  0.0688],
        [ 0.2278, -0.0096, -0.3427,  ...,  0.5979,  0.2396,  0.4793]])


In [ ]:
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters())

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds,y):
  rounded_preds=torch.round(torch.sigmoid(preds))
  correct=(rounded_preds == y).float()
  acc=correct.sum() / len(correct)
  return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text
        label=batch.label
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze(1)  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions,batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()

        # lr_schedule.step(epoch_loss)    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model,iterator,criterion):
  #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
          
            #retrieve text and no. of words
            text, text_lengths = batch.text
            label=batch.label
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze(1)
            
            #compute loss and accuracy
            loss = criterion(predictions,batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            # lr_schedule.step(epoch_loss)
            
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), dirs+'saved_weights_1.pt')
    
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

	Train Loss: 0.584 | Train Acc: 68.65%
	 Val. Loss: 0.486 |  Val. Acc: 77.97%
	Train Loss: 0.450 | Train Acc: 79.44%
	 Val. Loss: 0.385 |  Val. Acc: 82.45%
	Train Loss: 0.358 | Train Acc: 84.65%
	 Val. Loss: 0.316 |  Val. Acc: 87.05%
	Train Loss: 0.293 | Train Acc: 87.91%
	 Val. Loss: 0.282 |  Val. Acc: 88.53%
	Train Loss: 0.249 | Train Acc: 90.09%
	 Val. Loss: 0.282 |  Val. Acc: 89.22%


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [Text.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "i can not go outside")

0.3573157787322998

In [ ]:
predict_sentiment(model, "and and")

0.4225904047489166

In [ ]:
predict_sentiment(model,"me ready to go at supermarket")

0.0160974208265543

In [ ]:
predict_sentiment(model,"having fever since 2 days")

0.9474338889122009

In [ ]:
predict_sentiment(model,"i am corona positive")

0.012427430599927902

In [ ]:
predict_sentiment(model,"need food")

0.44335901737213135